### Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
application_df = application_df.drop(['EIN', 'NAME'], axis = 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_oh_df = application_df.merge(encode_df, left_index = True, right_index=True)
application_oh_df = application_oh_df.drop(application_cat, axis = 1)

In [11]:
application_oh_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = application_oh_df['IS_SUCCESSFUL'].values
X = application_oh_df.drop('IS_SUCCESSFUL', axis = 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1735)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### First Model Test: More neurons

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_input = len(X_train[0])

hidden_nodes_l1 = 10

hidden_nodes_l2 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units = hidden_nodes_l1, input_dim = n_input, activation = "relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l2, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                440       
                                                                 
 dense_1 (Dense)             (None, 7)                 77        
                                                                 
 dense_2 (Dense)             (None, 1)                 8         
                                                                 
Total params: 525
Trainable params: 525
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [19]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#checkpoint path and filenames
checkpoint_path = "checkpoints/moreweights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 'epoch',
    period = 5
)

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks = [cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 922us/step - loss: 0.5963 - accuracy: 0.7102
Epoch 2/100
804/804 [==============================] - 1s 908us/step - loss: 0.5640 - accuracy: 0.7266
Epoch 3/100
804/804 [==============================] - 1s 912us/step - loss: 0.5591 - accuracy: 0.7298
Epoch 4/100
804/804 [==============================] - 1s 913us/step - loss: 0.5574 - accuracy: 0.7300
Epoch 5/100
767/804 [===========================>..] - ETA: 0s - loss: 0.5558 - accuracy: 0.7294
Epoch 5: saving model to checkpoints\moreweights.05.hdf5
804/804 [==============================] - 1s 950us/step - loss: 0.5551 - accuracy: 0.7299
Epoch 6/100
804/804 [==============================] - 1s 976us/step - loss: 0.5541 - accuracy: 0.7301
Epoch 7/100
804/804 [==============================] - 1s 935us/step - loss: 0.5532 - accuracy: 0.7304
Epoch 8/100
804/804 [==============================] - 1s 969us/step - loss: 0.5525 - accuracy: 0.7308
Epoch 9/100
804/804 [=============

804/804 [==============================] - 1s 920us/step - loss: 0.5423 - accuracy: 0.7352
Epoch 64/100
804/804 [==============================] - 1s 912us/step - loss: 0.5423 - accuracy: 0.7345
Epoch 65/100
777/804 [===========================>..] - ETA: 0s - loss: 0.5412 - accuracy: 0.7356
Epoch 65: saving model to checkpoints\moreweights.65.hdf5
804/804 [==============================] - 1s 924us/step - loss: 0.5420 - accuracy: 0.7349
Epoch 66/100
804/804 [==============================] - 1s 914us/step - loss: 0.5420 - accuracy: 0.7340
Epoch 67/100
804/804 [==============================] - 1s 927us/step - loss: 0.5422 - accuracy: 0.7348
Epoch 68/100
804/804 [==============================] - 1s 914us/step - loss: 0.5422 - accuracy: 0.7344
Epoch 69/100
804/804 [==============================] - 1s 917us/step - loss: 0.5420 - accuracy: 0.7337
Epoch 70/100
754/804 [===========================>..] - ETA: 0s - loss: 0.5430 - accuracy: 0.7339
Epoch 70: saving model to checkpoints\morewe

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5626 - accuracy: 0.7221 - 278ms/epoch - 1ms/step
Loss: 0.5626145005226135, Accuracy: 0.7220991253852844


In [22]:
nn.save("moreneurons.h5")

### Second Try: sigmoid activations

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_input = len(X_train[0])

hidden_nodes_l1 = 7

hidden_nodes_l2 = 5

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units = hidden_nodes_l1, input_dim = n_input, activation = "sigmoid")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units = hidden_nodes_l2, activation = "sigmoid"))

# Output layer
nn2.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 7)                 308       
                                                                 
 dense_4 (Dense)             (None, 5)                 40        
                                                                 
 dense_5 (Dense)             (None, 1)                 6         
                                                                 
Total params: 354
Trainable params: 354
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn2.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [25]:
#checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok = True)
checkpoint_path = "checkpoints/sigmoidweights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 'epoch',
    period = 5
)

In [26]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100, callbacks = [cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 928us/step - loss: 0.6267 - accuracy: 0.6934
Epoch 2/100
804/804 [==============================] - 1s 908us/step - loss: 0.5770 - accuracy: 0.7243
Epoch 3/100
804/804 [==============================] - 1s 914us/step - loss: 0.5727 - accuracy: 0.7264
Epoch 4/100
804/804 [==============================] - 1s 920us/step - loss: 0.5690 - accuracy: 0.7261
Epoch 5/100
782/804 [============================>.] - ETA: 0s - loss: 0.5647 - accuracy: 0.7271
Epoch 5: saving model to checkpoints\sigmoidweights.05.hdf5
804/804 [==============================] - 1s 920us/step - loss: 0.5651 - accuracy: 0.7267
Epoch 6/100
804/804 [==============================] - 1s 971us/step - loss: 0.5621 - accuracy: 0.7281
Epoch 7/100
804/804 [==============================] - 1s 888us/step - loss: 0.5601 - accuracy: 0.7296
Epoch 8/100
804/804 [==============================] - 1s 919us/step - loss: 0.5586 - accuracy: 0.7300
Epoch 9/100
804/804 [==========

804/804 [==============================] - 1s 940us/step - loss: 0.5470 - accuracy: 0.7334
Epoch 64/100
804/804 [==============================] - 1s 905us/step - loss: 0.5470 - accuracy: 0.7337
Epoch 65/100
761/804 [===========================>..] - ETA: 0s - loss: 0.5464 - accuracy: 0.7338
Epoch 65: saving model to checkpoints\sigmoidweights.65.hdf5
804/804 [==============================] - 1s 940us/step - loss: 0.5468 - accuracy: 0.7328
Epoch 66/100
804/804 [==============================] - 1s 928us/step - loss: 0.5469 - accuracy: 0.7337
Epoch 67/100
804/804 [==============================] - 1s 909us/step - loss: 0.5468 - accuracy: 0.7327
Epoch 68/100
804/804 [==============================] - 1s 933us/step - loss: 0.5467 - accuracy: 0.7323
Epoch 69/100
804/804 [==============================] - 1s 936us/step - loss: 0.5466 - accuracy: 0.7334
Epoch 70/100
791/804 [============================>.] - ETA: 0s - loss: 0.5472 - accuracy: 0.7332
Epoch 70: saving model to checkpoints\sig

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5620 - accuracy: 0.7282 - 277ms/epoch - 1ms/step
Loss: 0.5620456337928772, Accuracy: 0.7281632423400879


In [28]:
nn2.save("sigmoid.h5")

### Adding another hidden layer

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_input = len(X_train[0])

hidden_nodes_l1 = 11

hidden_nodes_l2 = 7

hidden_nodes_l3 = 5

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units = hidden_nodes_l1, input_dim = n_input, activation = "sigmoid")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units = hidden_nodes_l2, activation = "relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units = hidden_nodes_l3, activation = "relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 11)                484       
                                                                 
 dense_7 (Dense)             (None, 7)                 84        
                                                                 
 dense_8 (Dense)             (None, 5)                 40        
                                                                 
 dense_9 (Dense)             (None, 1)                 6         
                                                                 
Total params: 614
Trainable params: 614
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn3.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [31]:
#checkpoint path and filenames
checkpoint_path = "checkpoints/sigmoidweights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq= 'epoch',
    period = 5
)

In [32]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=100, callbacks = [cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 959us/step - loss: 0.6254 - accuracy: 0.6765
Epoch 2/100
804/804 [==============================] - 1s 949us/step - loss: 0.5844 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 1s 951us/step - loss: 0.5751 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 1s 935us/step - loss: 0.5720 - accuracy: 0.7301
Epoch 5/100
787/804 [============================>.] - ETA: 0s - loss: 0.5701 - accuracy: 0.7312
Epoch 5: saving model to checkpoints\sigmoidweights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5704 - accuracy: 0.7305
Epoch 6/100
804/804 [==============================] - 1s 935us/step - loss: 0.5697 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 1s 954us/step - loss: 0.5664 - accuracy: 0.7316
Epoch 8/100
804/804 [==============================] - 1s 953us/step - loss: 0.5619 - accuracy: 0.7314
Epoch 9/100
804/804 [============

804/804 [==============================] - 1s 1ms/step - loss: 0.5433 - accuracy: 0.7358
Epoch 64/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7352
Epoch 65/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5428 - accuracy: 0.7346
Epoch 65: saving model to checkpoints\sigmoidweights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5430 - accuracy: 0.7345
Epoch 66/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5430 - accuracy: 0.7353
Epoch 67/100
804/804 [==============================] - 1s 974us/step - loss: 0.5431 - accuracy: 0.7344
Epoch 68/100
804/804 [==============================] - 1s 966us/step - loss: 0.5431 - accuracy: 0.7360
Epoch 69/100
804/804 [==============================] - 1s 941us/step - loss: 0.5429 - accuracy: 0.7345
Epoch 70/100
788/804 [============================>.] - ETA: 0s - loss: 0.5432 - accuracy: 0.7356
Epoch 70: saving model to checkpoints\sigmoidweig

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5634 - accuracy: 0.7254 - 273ms/epoch - 1ms/step
Loss: 0.5634452104568481, Accuracy: 0.7253644466400146


In [34]:
nn3.save("thirdlayer.h5")